<a href="https://colab.research.google.com/github/ayseakbaba/Cornac-vs-Surprise---SVD-NMF-BaseLine/blob/main/Cornac_vs_Surprise_SVD%2CNMF%2CBaseLine_Son_Versiyon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


##SURPRISE KÜTÜPHANESİ


---



#1. Kütüphanenin Eklenmesi

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357271 sha256=5aef150f611fc039005ba3fd9902992ac9206bc62fa47f28c2e64bacc1ea1218
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


#2. Importların Tanımlanması

In [ ]:
import os
import pandas as pd
import numpy as np
from surprise import Dataset, SVD, NMF, BaselineOnly, Reader
from surprise import accuracy
from sklearn.model_selection import train_test_split
from collections import defaultdict
from google.colab import drive
drive.mount('/content/myDrive')

Mounted at /content/myDrive


#3. Verisetinin Alınması ve Split Edilmesi

In [ ]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']

In [ ]:
data = pd.read_csv('/content/myDrive/MyDrive/datasets/ml-100k/u.data', sep='\t', encoding='utf-8', header=None, names=column_names)

In [ ]:
data['timestamp'] = data['timestamp'].apply(lambda x: f'{x:.0f}')

In [ ]:
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
trainset, testset = train_test_split(data[['user_id', 'item_id', 'rating']], test_size=0.2)

In [ ]:
type(trainset)

pandas.core.frame.DataFrame

#4. Split Edilmiş Verinin Surprise Kütüphanesine Uygun Hale Getirilmesi

In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
train_data_s = Dataset.load_from_df(trainset, reader)
test_data_s = Dataset.load_from_df(testset, reader)

In [ ]:
# Trainset'i oluşturan kod
trainset_surprise = train_data_s.build_full_trainset()

# Testset'i oluşturan kod
testset_surprise = test_data_s.build_full_trainset().build_testset()


In [ ]:
type(trainset_surprise)

surprise.trainset.Trainset

In [ ]:
type(testset_surprise)

list

#5. Modellerin Oluşturulması

In [ ]:
svd_surprise = SVD(n_factors=150, n_epochs=30, lr_all=0.001, reg_all=0.02, verbose=True)

In [ ]:
nmf_surprise = NMF(n_factors=50, n_epochs=30,reg_pu=0.02, reg_qi=0.02,verbose=True)

In [ ]:
baseLineOnly_surprise = BaselineOnly({'n_epochs':50, 'reg_u':0.02, 'reg_i':0.02},verbose=True)

#6. Modellerin Eğitilmesi

In [ ]:
svd_surprise.fit(trainset_surprise)
nmf_surprise.fit(trainset_surprise)
baseLineOnly_surprise.fit(trainset_surprise)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20


In [ ]:
testset_surprise = [tuple(x) for x in testset.to_records(index=False)]



---



---

##CORNAC KÜTÜPHANESİ


---




#1. Kütüphanenin Eklenmesi

In [ ]:
!pip install cornac

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 84.4 MB/s eta 0:00:00


#2. Importların Tanımlanması

In [ ]:
import cornac as cn
from sklearn.model_selection import train_test_split
from cornac.utils import cache
import numpy as np

#3. Split Edilmiş Verinin Cornac Kütüphanesine Uygun Hale Getirilmesi

In [ ]:
train_data_c = np.array(trainset)
test_data_c = np.array(testset)

In [ ]:
ratio_split = cn.eval_methods.RatioSplit(
    data=train_data_c, test_data=test_data_c,
    rating_threshold=4.0,
    exclude_unknowns=True,
    verbose=True
)

rating_threshold = 4.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1629
Number of ratings = 64000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1629
Number of ratings = 15960
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1629


In [ ]:
type(ratio_split)

cornac.eval_methods.ratio_split.RatioSplit

#5. Modellerin Oluşturulması

In [ ]:
svd_cornac = cn.models.SVD(k=150, max_iter=30, learning_rate=0.01, lambda_reg=0.02, verbose=True)
nmf_cornac = cn.models.NMF(k=50, max_iter=30, learning_rate=0.01, lambda_reg=0.02, verbose=True)
baselineOnly_cornac = cn.models.BaselineOnly(max_iter=50, learning_rate=0.01, lambda_reg=0.02, verbose=True)

#6. Modellerin Eğitimi

In [ ]:
cn.Experiment(eval_method=ratio_split, models=[svd_cornac, nmf_cornac, baselineOnly_cornac]).run()


[SVD] Training started!


  0%|          | 0/30 [00:00<?, ?it/s]

Optimization finished!

[SVD] Evaluation started!


Rating:   0%|          | 0/15960 [00:00<?, ?it/s]

Ranking:   0%|          | 0/939 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Optimization finished!

[BaselineOnly] Evaluation started!


Rating:   0%|          | 0/15960 [00:00<?, ?it/s]

Ranking:   0%|          | 0/939 [00:00<?, ?it/s]


TEST:
...
             |    MAE |   RMSE | Precision@20 | Recall@20 | Train (s) | Test (s)
------------ + ------ + ------ + ------------ + --------- + --------- + --------
SVD          | 0.7421 | 0.8989 |       0.0512 |    0.1000 |    0.4521 |   1.5468
NMF          | 1.3754 | 1.6688 |       0.0112 |    0.0196 |    0.7663 |   3.6390
BaselineOnly | 0.7585 | 0.9165 |       0.0277 |    0.0561 |    0.2197 |   1.0781





---

#PERFORMANS TESTLERİ


---



In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import random
import os

In [ ]:
# 1. Precision@k
def precision_at_k(predictions, k=10):
    top_k = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k[uid].append((iid, est))

    precision_scores = []
    for uid, user_ratings in top_k.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        relevant_items = set([iid for iid, true_r in user_ratings if true_r >= 4])  # Doğru öğe (rating >= 4)
        recommended_items = set([iid for iid, _ in user_ratings[:k]])  # İlk k öğe

        precision = len(relevant_items & recommended_items) / k
        precision_scores.append(precision)

    return np.mean(precision_scores)


In [ ]:
# 2. Recall@k
def recall_at_k(predictions, k=10):
    top_k = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k[uid].append((iid, est))

    recall_scores = []
    for uid, user_ratings in top_k.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        relevant_items = set([iid for iid, true_r in user_ratings if true_r >= 4])  # Doğru öğe
        recommended_items = set([iid for iid, _ in user_ratings[:k]])  # İlk k öğe

        recall = len(relevant_items & recommended_items) / len(relevant_items) if relevant_items else 0
        recall_scores.append(recall)

    return np.mean(recall_scores)


In [ ]:
# 3. F1-Score@k
def f1_score_at_k(predictions, k=10):
    precision = precision_at_k(predictions, k)
    recall = recall_at_k(predictions, k)

    if precision + recall == 0:
        return 0

    return 2 * (precision * recall) / (precision + recall)

In [ ]:
# 4. Mean Average Precision (MAP)
def average_precision_at_k(predictions, k=10):
    top_k = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k[uid].append((iid, est))

    ap_scores = []
    for uid, user_ratings in top_k.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        relevant_items = set([iid for iid, true_r in user_ratings if true_r >= 4])
        recommended_items = [iid for iid, _ in user_ratings[:k]]

        ap = 0
        num_relevant = 0
        for rank, item in enumerate(recommended_items, start=1):
            if item in relevant_items:
                num_relevant += 1
                ap += num_relevant / rank
        ap_scores.append(ap / k if relevant_items else 0)

    return np.mean(ap_scores)

In [ ]:
# 5. Normalized Discounted Cumulative Gain (NDCG)
def ndcg_at_k(predictions, k=10):
    def dcg_at_k(relevant_items, recommended_items, k):
        dcg = 0
        for i, item in enumerate(recommended_items[:k]):
            if item in relevant_items:
                dcg += 1 / np.log2(i + 2)  # log2(i + 2) çünkü sıralama 1'den başlar
        return dcg

    top_k = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k[uid].append((iid, est))

    ndcg_scores = []
    for uid, user_ratings in top_k.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        relevant_items = set([iid for iid, true_r in user_ratings if true_r >= 4])  # Doğru öğeler
        recommended_items = [iid for iid, _ in user_ratings[:k]]

        ideal_dcg = dcg_at_k(relevant_items, recommended_items, k)
        dcg = dcg_at_k(relevant_items, recommended_items, k)

        ndcg = dcg / ideal_dcg if ideal_dcg != 0 else 0
        ndcg_scores.append(ndcg)

    return np.mean(ndcg_scores)

In [ ]:
# 6. Coverage
def coverage(predictions, total_items):
    recommended_items = set([iid for _, iid, _, _, _ in predictions])
    return len(recommended_items) / total_items

In [ ]:
# 7. Diversity
def diversity(predictions, k=10):
    top_k = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k[uid].append((iid, est))

    diversity_scores = []
    for uid, user_ratings in top_k.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        recommended_items = [iid for iid, _ in user_ratings[:k]]

        # Öğeler arasındaki çeşitliliği hesaplamak için benzerlik yerine basitçe ID'ler arası farklılık kullanacağız
        item_pairs = [(i1, i2) for i1 in recommended_items for i2 in recommended_items if i1 != i2]
        diversity_score = 0
        for i1, i2 in item_pairs:
            # Öğeler arasındaki farklılığı saymak için ID'leri rastgele atayabiliriz (sadece örnek olarak)
            diversity_score += random.random()  # Öneri sisteminde çeşitlilik hesaplamak için basit bir yaklaşım
        diversity_scores.append(diversity_score / len(item_pairs) if item_pairs else 0)

    return np.mean(diversity_scores)

In [ ]:
# 8. Mean Reciprocal Rank (MRR)
def mrr(predictions):
    ranks = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        if true_r >= 4:  # Doğru öğe
            ranks[uid].append(iid)

    mrr_scores = []
    for uid, user_ranks in ranks.items():
        if user_ranks:
            rank = user_ranks.index(user_ranks[0]) + 1  # İlk doğru öğenin sırası
            mrr_scores.append(1 / rank)

    return np.mean(mrr_scores)

In [ ]:
# 9. Average Precision at Top-N (APLT)
def aplt_at_k(predictions, k=10):
    top_k = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k[uid].append((iid, est))

    aplt_scores = []
    for uid, user_ratings in top_k.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        recommended_items = [iid for iid, _ in user_ratings[:k]]

        relevant_items = set([iid for iid, true_r in user_ratings if true_r >= 4])
        ap = 0
        num_relevant = 0
        for rank, item in enumerate(recommended_items, start=1):
            if item in relevant_items:
                num_relevant += 1
                ap += num_relevant / rank
        aplt_scores.append(ap / k if relevant_items else 0)

    return np.mean(aplt_scores)


In [ ]:
# 10. Evaluate Metrics
def evaluate_top_n_metrics(predictions, k=10, total_items=None):
    # Hesaplama işlemleri
    f1 = f1_score_at_k(predictions, k)
    map_score = average_precision_at_k(predictions, k)
    ndcg = ndcg_at_k(predictions, k)
    coverage_score = coverage(predictions, total_items)
    diversity_score = diversity(predictions, k)
    mrr_score = mrr(predictions)
    aplt_score = aplt_at_k(predictions, k)  # APLT metrik hesaplaması

    metrics = {
        'Precision@k': precision_at_k(predictions, k),
        'Recall@k': recall_at_k(predictions, k),
        'F1@k': f1,
        'MAP@k': map_score,
        'NDCG@k': ndcg,
        'Coverage': coverage_score,
        'Diversity': diversity_score,
        'MRR': mrr_score,
        'APLT@k': aplt_score
    }

    return metrics

In [ ]:
# Evaluation for Cornac
def evaluate_cornac_model(model, test_data_c, k=5, total_items=1682):
    predictions = []
    for uid, iid, true_r in test_data_c:
        prediction = model.rate(uid, iid)  # rate fonksiyonunu doğru parametrelerle çağırdık
        predictions.append([uid, iid, true_r, prediction, []])  # format: [user_id, item_id, true_rating, predicted_rating]
    return evaluate_top_n_metrics(predictions, k, total_items)

In [ ]:
# Evaluation for Surprise
def evaluate_surprise_model(model, test_data, k=5, total_items=1682):
    prediction = model.test(test_data)
    return evaluate_top_n_metrics(prediction, k, total_items)

In [ ]:
def remap_ids(test_data, user_mapping=None, item_mapping=None):
    """
    Verilen test veri kümesindeki kullanıcı ve öğe ID'lerini modelin eğitim verisine uygun hale getirir.
    Eğer kullanıcı ve öğe haritalamaları mevcutsa, bu haritalamaları kullanarak ID'leri yeniden eşler.
    """
    if user_mapping is None:
        user_mapping = {uid: idx for idx, uid in enumerate(np.unique([entry[0] for entry in test_data]))}
    if item_mapping is None:
        item_mapping = {iid: idx for idx, iid in enumerate(np.unique([entry[1] for entry in test_data]))}

    # Test verisindeki kullanıcı ve öğe ID'lerini yeniden eşleyelim
    remapped_test_data = [(user_mapping[uid], item_mapping[iid], true_r) for uid, iid, true_r in test_data]
    return remapped_test_data, user_mapping, item_mapping

In [ ]:
# 10. Evaluate Metrics with Harmonic Mean
def evaluate_top_n_metrics_with_harmonic_mean(predictions, k=10, total_items=None):
    # Hesaplama işlemleri
    precision = precision_at_k(predictions, k)
    recall = recall_at_k(predictions, k)
    f1 = f1_score_at_k(predictions, k)
    map_score = average_precision_at_k(predictions, k)
    ndcg = ndcg_at_k(predictions, k)
    coverage_score = coverage(predictions, total_items)
    diversity_score = diversity(predictions, k)
    mrr_score = mrr(predictions)
    aplt_score = aplt_at_k(predictions, k)  # APLT metrik hesaplaması

    # Tüm metriklerin bir listesi
    metrics_list = [
        precision,
        recall,
        f1,
        map_score,
        ndcg,
        coverage_score,
        diversity_score,
        mrr_score,
        aplt_score
    ]

    # Harmonik ortalama hesaplama
    harmonic_mean = len(metrics_list) / sum(1/x for x in metrics_list if x > 0)  # Sıfır olmayan değerler ile

    # Metrikleri dictionary formatında toplamak
    metrics = {
        'Precision@k': precision,
        'Recall@k': recall,
        'F1@k': f1,
        'MAP@k': map_score,
        'NDCG@k': ndcg,
        'Coverage': coverage_score,
        'Diversity': diversity_score,
        'MRR': mrr_score,
        'APLT@k': aplt_score,
        'HarmonicMean': harmonic_mean  # Harmonik ortalama ekleniyor
    }

    return metrics

In [ ]:
def evaluate_all_algorithms_with_harmonic_mean(cornac_algorithms, surprise_algorithms, test_data_c, test_data, k=10, total_items=None):
    results = []

    # Cornac algoritmalarını değerlendir
    for algo in cornac_algorithms:
        predictions = []
        # Test verisini remap ettiğimizden emin olun
        test_data_c_remapped, user_mapping, item_mapping = remap_ids(test_data_c)
        for uid, iid, true_r in test_data_c_remapped:
            prediction = algo.rate(uid, iid)  # rate fonksiyonunu doğru parametrelerle çağırıyoruz
            predictions.append([uid, iid, true_r, prediction, []])

        algo_metrics = evaluate_top_n_metrics_with_harmonic_mean(predictions, k, total_items)
        algo_df = pd.DataFrame([algo_metrics])  # Burada dict'i DataFrame'e dönüştür
        algo_name = f"{algo.__class__.__name__}_Cornac"
        algo_df['Algorithm'] = algo_name  # Algoritma adı ekleniyor
        results.append(algo_df)

    # Surprise algoritmalarını değerlendir
    for algo in surprise_algorithms:
        predictions = algo.test(test_data)
        algo_metrics = evaluate_top_n_metrics_with_harmonic_mean(predictions, k, total_items)
        algo_df = pd.DataFrame([algo_metrics])  # Burada dict'i DataFrame'e dönüştür
        algo_name = f"{algo.__class__.__name__}_Surprise"
        algo_df['Algorithm'] = algo_name  # Algoritma adı ekleniyor
        results.append(algo_df)

    # Sonuçları DataFrame'e çevir
    metrics_df = pd.concat(results, axis=0, ignore_index=True)
    return metrics_df

In [ ]:
# Algoritmalar
cornac_algorithms = [svd_cornac, nmf_cornac, baselineOnly_cornac]
surprise_algorithms = [svd_surprise, nmf_surprise, baseLineOnly_surprise]

# Test veri setlerini ve total item sayısını belirleyin
test_data_c = test_data_c
test_data = testset_surprise
total_items = len(movie_dict)  # veya başka bir total item sayısı

# Algoritmaları değerlendirin
metrics_df = evaluate_all_algorithms_with_harmonic_mean(cornac_algorithms, surprise_algorithms, test_data_c, test_data, k=10, total_items=total_items)

# Sonuçları yazdırın
print(metrics_df)

   Precision@k  Recall@k      F1@k     MAP@k    NDCG@k  Coverage  Diversity  \
0     0.340403  0.685083  0.454817  0.340403  0.735949   0.82283   0.494872   
1     0.830329  0.676470  0.745544  0.830329  1.000000   0.82283   0.496197   
2     0.336479  0.629575  0.438565  0.336479  0.687169   0.82283   0.497329   
3     0.338600  0.659552  0.447476  0.338600  0.710498   0.82283   0.495914   
4     0.831283  0.672576  0.743555  0.831283  0.998940   0.82283   0.492935   
5     0.382821  0.656213  0.483549  0.382821  0.718982   0.82283   0.494961   

   MRR    APLT@k  HarmonicMean              Algorithm  
0  1.0  0.340403      0.498166             SVD_Cornac  
1  1.0  0.830329      0.771655             NMF_Cornac  
2  1.0  0.336479      0.487390    BaselineOnly_Cornac  
3  1.0  0.338600      0.493145           SVD_Surprise  
4  1.0  0.831283      0.770176           NMF_Surprise  
5  1.0  0.382821      0.527703  BaselineOnly_Surprise  


In [ ]:
# Sonuçları Excel dosyası olarak kaydet
metrics_df.to_excel('algorithm_comparison_results.xlsx', index=False)



---

#FİLM ÖNERİSİNDE BULUNMA


---



In [ ]:
data_folder = os.path.expanduser('/content/myDrive/MyDrive/datasets/ml-100k/')

# Film bilgilerini içeren dosyayı pandas ile yükleyin
movie_info_path = os.path.join(data_folder, 'u.item')
movie_info = pd.read_csv(movie_info_path, sep='|', encoding='latin-1', header=None)
movie_info.columns = ['movie_id', 'movie_title'] + [str(i) for i in range(22)]
movie_dict = dict(zip(movie_info['movie_id'], movie_info['movie_title']))

In [ ]:
def get_recommendations_for_cornac(algo, user_id, movie_dict, n=5):
    # Kullanıcının önerilen filmlerini alın
    predicted_movie_ids = algo.recommend(user_id, n)
    movieList = []

    # Önerilen filmler ve puanları
    for predict in predicted_movie_ids:
        movie_id = int(predict)
        score = np.round(algo.rate(int(user_id), movie_id))  # Kullanıcı ve film için puan al
        movie_title = movie_dict.get(movie_id, 'Unknown Movie')  # Filmi movie_dict'den al
        movieList.append((movie_title, score))  # Film adı ve puanını tuple olarak ekle

    return movieList

In [ ]:
def get_recommendations_for_surprise(algo, user_id, item_ids, movie_dict, top_n=5):
    # Kullanıcı için film tahminlerini al
    predictions = [algo.predict(user_id, iid) for iid in item_ids]
    predictions.sort(key=lambda x: x.est, reverse=True)  # Puanlara göre sırala
    top_predictions = predictions[:top_n]

    # En yüksek puanlı ilk N filmi döndür
    return [(movie_dict.get(int(pred.iid), 'Unknown Movie'), round(pred.est)) for pred in top_predictions]


In [ ]:
user_id = str(1)
item_ids = [iid for (uid, iid, _) in testset_surprise if user_id == user_id]  # Kullanıcının izleyebileceği filmleri seç

In [ ]:
print("SVD Algoritmasına Göre Öneriler:")
print(get_recommendations_for_surprise(svd_surprise, user_id, item_ids, movie_dict))

print("\nNMF Algoritmasına Göre Öneriler:")
print(get_recommendations_for_surprise(nmf_surprise, user_id, item_ids, movie_dict))

print("\nSlopeOne Algoritmasına Göre Öneriler:")
print(get_recommendations_for_surprise(baseLineOnly_surprise, user_id, item_ids, movie_dict))


SVD Algoritmasına Göre Öneriler:
[('Shawshank Redemption, The (1994)', 5), ('Shawshank Redemption, The (1994)', 5), ('Shawshank Redemption, The (1994)', 5), ('Shawshank Redemption, The (1994)', 5), ('Shawshank Redemption, The (1994)', 5)]

NMF Algoritmasına Göre Öneriler:
[('Empire Strikes Back, The (1980)', 4), ('Rock, The (1996)', 4), ('Mimic (1997)', 4), ('Ciao, Professore! (1993)', 4), ('Fried Green Tomatoes (1991)', 4)]

SlopeOne Algoritmasına Göre Öneriler:
[('Santa with Muscles (1996)', 5), ('Saint of Fort Washington, The (1993)', 5), ('Delta of Venus (1994)', 5), ('Pather Panchali (1955)', 5), ('Pather Panchali (1955)', 5)]


In [ ]:
print("SVD Algoritmasına Göre Öneriler:")
print(get_recommendations_for_cornac(svd_cornac, 1, movie_dict))

print("\nNMF Algoritmasına Göre Öneriler:")
print(get_recommendations_for_cornac(nmf_cornac, 1, movie_dict))

print("\nBaselineOnly Algoritmasına Göre Öneriler:")
print(get_recommendations_for_cornac(baselineOnly_cornac, 1, movie_dict))

SVD Algoritmasına Göre Öneriler:
[('Star Wars (1977)', 2.0), ('Return of the Jedi (1983)', 2.0), ('Swingers (1996)', 4.0), ('Monty Python and the Holy Grail (1974)', 3.0), ('Blade Runner (1982)', 4.0)]

NMF Algoritmasına Göre Öneriler:
[('M (1931)', 5.0), ('World of Apu, The (Apur Sansar) (1959)', 5.0), ('Godfather, The (1972)', 5.0), ('Hugo Pool (1997)', 2.0), ('Mina Tannenbaum (1994)', 4.0)]

BaselineOnly Algoritmasına Göre Öneriler:
[('World of Apu, The (Apur Sansar) (1959)', 3.0), ('Pather Panchali (1955)', 3.0), ('Shawshank Redemption, The (1994)', 3.0), ('Wrong Trousers, The (1993)', 3.0), ('Close Shave, A (1995)', 3.0)]
